In [1]:
!pip install selenium beautifulsoup4 requests



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [13]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time

# Setup Chrome options (VISIBLE MODE)
options = Options()
# options.add_argument("--headless")  # <-- remove or comment this line to see the browser
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")

# Setup ChromeDriver service with your path
chrome_driver_path = "/Users/kowshik/Documents/INTERNSHIP_PYTHON_ENG/chromedriver-mac-arm64/chromedriver"
service = Service(executable_path=chrome_driver_path)

# Create WebDriver instance
driver = webdriver.Chrome(service=service, options=options)

# Step 1: Visit the main page
driver.get("https://rera.odisha.gov.in/projects/project-list")
time.sleep(5)

# Step 2: Get the first 6 "View Details" project links
soup = BeautifulSoup(driver.page_source, "html.parser")
project_links = []
for a_tag in soup.select('a[title="View Details"]')[:6]:
    link = a_tag.get("href")
    if link:
        full_link = "https://rera.odisha.gov.in" + link
        project_links.append(full_link)

# Step 3: Visit each link and scrape details
projects_data = []

for url in project_links:
    driver.get(url)
    time.sleep(5)
    soup = BeautifulSoup(driver.page_source, "html.parser")

    data = {}

    try:
        data['RERA Regd. No'] = soup.select_one("span#ContentPlaceHolder1_lblRegistrationNumber").text.strip()
        data['Project Name'] = soup.select_one("span#ContentPlaceHolder1_lblProjectName").text.strip()
    except:
        data['RERA Regd. No'] = data['Project Name'] = 'Not found'

    # Try to click Promoter Details tab
    try:
        promoter_tab = driver.find_element("xpath", "//a[text()='Promoter Details']")
        promoter_tab.click()
        time.sleep(2)
        soup = BeautifulSoup(driver.page_source, "html.parser")
    except:
        pass

    try:
        data['Promoter Name'] = soup.select_one("span#ContentPlaceHolder1_lblPromoterName").text.strip()
    except:
        data['Promoter Name'] = 'Not found'

    try:
        data['Registered Office Address'] = soup.select_one("span#ContentPlaceHolder1_lblRegAddress").text.strip()
    except:
        data['Registered Office Address'] = 'Not found'

    try:
        data['GST No'] = soup.select_one("span#ContentPlaceHolder1_lblGSTNumber").text.strip()
    except:
        data['GST No'] = 'Not found'

    data['Detail Page'] = url
    projects_data.append(data)

# Step 4: Display the result
for i, project in enumerate(projects_data, 1):
    print(f"\nProject {i}")
    for key, value in project.items():
        print(f"{key}: {value}")




In [15]:
#!/usr/bin/env python3
"""
rera_scraper.py

Dependencies:
    pip install selenium beautifulsoup4

Chromedriver (v137) path:
    /Users/kowshik/Documents/INTERNSHIP_PYTHON_ENG/chromedriver-mac-arm64/chromedriver
"""

import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup

# 1) Configure ChromeDriver
chromedriver_path = "/Users/kowshik/Documents/INTERNSHIP_PYTHON_ENG/chromedriver-mac-arm64/chromedriver"
service = Service(executable_path=chromedriver_path)

options = Options()
# comment out the next line if you want to watch the browser
# options.add_argument("--headless")
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")

driver = webdriver.Chrome(service=service, options=options)

# 2) Load project list page
driver.get("https://rera.odisha.gov.in/projects/project-list")
time.sleep(5)  # wait for JavaScript

# 3) Grab exactly the first 6 “View Details” links
buttons = driver.find_elements(By.XPATH, "//a[contains(text(),'View Details')]")[:6]
project_links = [btn.get_attribute("href") for btn in buttons]

projects = []

# 4) Visit each detail page and scrape
for url in project_links:
    driver.get(url)
    time.sleep(3)
    soup = BeautifulSoup(driver.page_source, "html.parser")

    # Prepare a dict for this project
    p = {"Detail Page": url}

    # a) General Info
    try:
        p["RERA Regd. No"]   = soup.select_one("span#ContentPlaceHolder1_lblRegistrationNumber").get_text(strip=True)
    except:
        p["RERA Regd. No"]   = "N/A"
    try:
        p["Project Name"]    = soup.select_one("span#ContentPlaceHolder1_lblProjectName").get_text(strip=True)
    except:
        p["Project Name"]    = "N/A"

    # b) Switch to Promoter Details tab
    try:
        driver.find_element(By.XPATH, "//a[text()='Promoter Details']").click()
        time.sleep(2)
        soup = BeautifulSoup(driver.page_source, "html.parser")
    except:
        pass

    # c) Promoter Details
    try:
        p["Promoter Name"]   = soup.select_one("span#ContentPlaceHolder1_lblPromoterName").get_text(strip=True)
    except:
        p["Promoter Name"]   = "N/A"
    try:
        p["Promoter Address"]= soup.select_one("span#ContentPlaceHolder1_lblRegAddress").get_text(strip=True)
    except:
        p["Promoter Address"]= "N/A"
    try:
        p["GST No"]          = soup.select_one("span#ContentPlaceHolder1_lblGSTNumber").get_text(strip=True)
    except:
        p["GST No"]          = "N/A"

    projects.append(p)

# 5) Tear down
driver.quit()

# 6) Output results
for i, proj in enumerate(projects, start=1):
    print(f"\n=== Project {i} ===")
    for k, v in proj.items():
        print(f"{k}: {v}")


WebDriverException: Message: unknown error: unsupported protocol
  (Session info: chrome=137.0.7151.41)
Stacktrace:
0   chromedriver                        0x0000000100c1c3e0 cxxbridge1$str$ptr + 2829900
1   chromedriver                        0x0000000100c146a8 cxxbridge1$str$ptr + 2797844
2   chromedriver                        0x0000000100751fbc cxxbridge1$string$len + 90140
3   chromedriver                        0x0000000100754f64 cxxbridge1$string$len + 102340
4   chromedriver                        0x00000001007daab0 cxxbridge1$string$len + 650000
5   chromedriver                        0x00000001007da044 cxxbridge1$string$len + 647332
6   chromedriver                        0x000000010078d3f8 cxxbridge1$string$len + 332888
7   chromedriver                        0x0000000100be0804 cxxbridge1$str$ptr + 2585200
8   chromedriver                        0x0000000100be3ad4 cxxbridge1$str$ptr + 2598208
9   chromedriver                        0x0000000100bc1dd8 cxxbridge1$str$ptr + 245